# Vector indexing for Image-appended Data on Google Colab

In [1]:
!pip install python-dotenv

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0a99fcd29ed6e24310dcaa55b1ab99d4006ff9003bce28e7fe2fb58863fca114
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00
time: 311 µs (started: 2024-01-12 02:43:13 +00:00)


In [5]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 16.8 MB/s eta 0:00:00
time: 8.85 s (started: 2024-01-12 02:43:13 +00:00)


In [6]:
import os
import json
from pathlib import Path
from pprint import pprint

from bs4 import BeautifulSoup

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

time: 1.11 s (started: 2024-01-12 02:43:22 +00:00)


In [27]:
data[13]

{'html': 'Elephant seals on San Nicolas Island beach. Courtesy of Steve Schwartz.\nFor more documents visit the digital archive at:\xa0http://calliope.cse.sc.edu/lonewoman/home\nAn official form of the United States government. Provided by Touchpoints\nDownload the official NPS app before your next visit',
 'title': 'Primary Source: Chapter 13 - Island of the Blue Dolphins (U.S. National Park Service)',
 'url': 'https://www.nps.gov/subjects/islandofthebluedolphins/primary13.htm'}

time: 2.92 ms (started: 2024-01-12 02:51:15 +00:00)


In [29]:
# file_path='./drive/MyDrive/vectordb/output_grca_clean.json'
file_path='./drive/MyDrive/vectordb/output_all_parks_clean_img.json'

data = json.loads(Path(file_path).read_text())
# Inspect visually the loaded data
data[0]
# Use Beautiful Soup to format html into txt
data_txt = []
for d in data:
    data_txt.append(
        "TITLE: "+d['title']+" \n"+
        "URL: "+d['url']+" \n"+
        "HTML: "+ BeautifulSoup(d['html']).get_text()
    )
    if "images" in d.keys():
        # print(d['images'])
        for k in d['images'].keys():
            data_txt.append("IMAGE URL: "+str(k)+"\nDESCRIPTION: "+d['images'][k])
# Inspect the formatted data
pprint(data_txt[0])

<ipython-input-29-96035e7fbd03>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  "HTML: "+ BeautifulSoup(d['html']).get_text()


('TITLE: Teacher Resources: Chapter 13 - Island of the Blue Dolphins (U.S. '
 'National Park Service) \n'
 'URL: https://www.nps.gov/subjects/islandofthebluedolphins/teacher-13.htm \n'
 'HTML: Opinion: Think about ways men and women on San Nicolas Island might '
 'have shared the division of labor. Do you agree or disagree with the custom '
 'that women in Karana’s tribe were not allowed to make weapons? Remember to '
 'consider what it might have been like to live in her particular society. Use '
 'reasons and information to support your point of view.\n'
 'Informative/explanatory: Describe the mating and breeding habits of northern '
 'elephant seals. Gather information from Voices from the Field to develop the '
 'topic. Draw evidence from informational text.\n'
 'Narrative: Imagine you are Karana watching a fight between two bull elephant '
 'seals. Write a narrative describing the battle. Include details about what '
 'you might see, hear, and smell.\n'
 'An official form of the U

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# Convert plain text to documents
docs = text_splitter.create_documents(data_txt)
# Inspect the documents
docs[3:10]
pprint(docs[2].to_json()['kwargs']['page_content'])

('TITLE: Teacher Resources: Chapter 11 - Island of the Blue Dolphins (U.S. '
 'National Park Service) \n'
 'URL: https://www.nps.gov/subjects/islandofthebluedolphins/teacher-11.htm \n'
 'HTML: Opinion: What factors did Karana consider in choosing the location for '
 'her new home? Would you have chosen the same location? Why or why not? Use '
 'evidence from the story to support your claim.\n'
 'Informative/explanatory: Describe some of the ways Karana protects her food '
 'and possessions from wild animals. Develop the topic with facts and other '
 'information related to the topic.\n'
 'Narrative: Karana plans to kill the wild dogs. Imagine you are a wild dog. '
 'How might you react when Karana approaches your cave? Use sensory details to '
 'convey experiences and events precisely.\n'
 'An official form of the United States government. Provided by Touchpoints\n'
 'Download the official NPS app before your next visit')
time: 18.8 s (started: 2024-01-12 02:53:49 +00:00)


In [31]:
# !ls drive/MyDrive/vectordb/output_grca_clean.json
!ls drive/MyDrive/vectordb

faiss_index_all_parks  output_all_parks_clean_img.json	output_grca_clean.json
faiss_index_grca       output_all_parks_clean.json
time: 301 ms (started: 2024-01-12 02:54:08 +00:00)


In [32]:
## Build the vector database
modelPath = "all-MiniLM-L6-v2"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings':False}

embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

time: 11.4 s (started: 2024-01-12 02:54:20 +00:00)


In [33]:
# db = FAISS.from_documents(docs, embeddings)
db = await FAISS.afrom_documents(docs, embeddings)

time: 8min 58s (started: 2024-01-12 02:54:31 +00:00)


In [34]:
db.save_local("faiss_index")
# db.save_local("faiss_index_all_parks")
db.save_local("faiss_index_all_parks_image")

time: 12.3 s (started: 2024-01-12 03:03:32 +00:00)
